In [1]:
import pandas as pd
import numpy as np
import time
import seaborn as sb
import matplotlib.pyplot as plt

In [2]:
df =  pd.read_csv("./dataset/creditcard.csv")

FileNotFoundError: [Errno 2] No such file or directory: './dataset/creditcard.csv'

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.Class.value_counts()

In [ ]:
fraud = df[df['Class']==1]
normal = df[df['Class']==0]

print(f"Shape of Fraudulant transactions: {fraud.shape}")
print(f"Shape of Non-Fraudulant transactions: {normal.shape}")

In [ ]:
pd.concat([fraud.Amount.describe(), normal.Amount.describe()], axis=1)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score

def print_score(label, prediction, train=True):
    if train:
        clf_report = pd.DataFrame(classification_report(label, prediction, output_dict=True))
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(label, prediction) * 100:.2f}%")
        print("_______________________________________________")
        print(f"Classification Report:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_train, prediction)}\n")
        
    elif train==False:
        clf_report = pd.DataFrame(classification_report(label, prediction, output_dict=True))
        print("Test Result:\n================================================")        
        print(f"Accuracy Score: {accuracy_score(label, prediction) * 100:.2f}%")
        print("_______________________________________________")
        print(f"Classification Report:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(label, prediction)}\n") 

In [ ]:
x = df.drop('Class', axis=1)
y = df.Class

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [ ]:
x_train.shape

In [ ]:
from xgboost import XGBClassifier

xgb_clf = XGBClassifier()
start = time.time()
xgb_clf.fit(x_train, y_train, eval_metric='aucpr')
stop = time.time()

print(f"Training time: {(stop - start) * 1000 }ms")

y_train_pred = xgb_clf.predict(x_train)
y_test_pred = xgb_clf.predict(x_test)

print_score(y_train, y_train_pred, train=True)
print_score(y_test, y_test_pred, train=False)

scores_dict = { }
scores_dict['XGBoost'] = {
        'Train': f1_score(y_train,y_train_pred),
        'Test': f1_score(y_test, y_test_pred),
}

In [ ]:
import shap

explainer = shap.Explainer(xgb_clf)
shap_values = explainer.shap_values(x_train)

In [ ]:
shap.initjs()

shap.summary_plot(shap_values, x_train)

In [ ]:
x = df[['V14', 'V4', 'V12', 'V10', 'Time', 'V8', 'V19', 'Amount', 'V3', 'V7', 'V26', 'V6', 'V5', 'V27', 'V15', 'V16', 'V20', 'V22', 'V11', 'V21']]
y = df.Class

In [ ]:
x.head()
x.to_csv('./dataset/filtered_csv')

In [ ]:
vals= np.abs(shap_values).mean(0)
feature_importance = pd.DataFrame(list(zip(x_train.columns,vals)),columns=['feature','importance'])
feature_importance.sort_values(by=['importance'],ascending=False,inplace=True)

feature_importance.head()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)


xgb_clf = XGBClassifier()
start = time.time()
xgb_clf.fit(x_train, y_train, eval_metric='aucpr')
stop = time.time()

print(f"Training time: {(stop - start) * 1000 }ms")

y_train_pred = xgb_clf.predict(x_train)
y_test_pred = xgb_clf.predict(x_test)

print_score(y_train, y_train_pred, train=True)
print_score(y_test, y_test_pred, train=False)

scores_dict = { }
scores_dict['XGBoost'] = {
        'Train': f1_score(y_train,y_train_pred),
        'Test': f1_score(y_test, y_test_pred),
}